# Capstone Project Part 3: Building a Streaming Platform Recommender Using AI Chatbot

## Contents
1. [Install packages, import libraries, API and set filepath](#Install-packages,-import-libraries,-API-and-set-filepath)
2. [Load the questions & answers CSV dataset](#Load-the-questions-&-answers-CSV-dataset)
3. [Build Index](#Build-Index)
4. [Train Questions Dataset](#Train-Questions-Dataset)
5. [Evaluation Questions Dataset](#Evaluation-Questions-Dataset)
6. [Initial Evaluation](#Initial-Evaluation)
7. [2nd Evaluation](#2nd-Evaluation)
8. [3rd Evaluation (Fine Tune)](#3rd-Evaluation-(Fine-Tune))

## Install packages, import libraries, API and set filepath

In [115]:
#install relevant packages to run the AI Chatbot
#!pip install llama_index==0.8.64
#!pip install openai
# !pip install spacy
# %pip install llama-index==0.8.64 pypdf sentence-transformers ragas openai 

   ---------------------------------------- 0.0/362.9 kB ? eta -:--:--
   ---------- ----------------------------- 92.2/362.9 kB 2.6 MB/s eta 0:00:01
   ------------------- -------------------- 174.1/362.9 kB 2.1 MB/s eta 0:00:01
   ---------------------------- ----------- 256.0/362.9 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------  358.4/362.9 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 362.9/362.9 kB 1.9 MB/s eta 0:00:00


In [4]:
#import relevant libraries as well as API key to run the AI Chatbot
import os
os.environ['OPENAI_API_KEY'] = "" # replace with your API key

from llama_index import Document, GPTVectorStoreIndex, ServiceContext, VectorStoreIndex
from llama_index.readers import BeautifulSoupWebReader, SimpleDirectoryReader
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator
from llama_index.llms.huggingface import HuggingFaceLLM

import openai

from pathlib import Path
from llama_index import download_loader

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness
from llama_index.response.notebook_utils import display_response
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

import random
import nest_asyncio

In [33]:
# set filepath to my data directory 
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, "./data")

## Load the questions & answers CSV dataset

In [36]:
#read and load the csv file into the model
PagedCSVReader = download_loader("PagedCSVReader")

loader = PagedCSVReader(encoding="utf-8")
docs = loader.load_data(file=Path('./data/questions_answers.csv')) 

## Build Index

With all the data loaded, we can construct the index for the chatbot. There are 4 types of indexing: Summary index, VectorStore Index, Tree Index and Keyword Table Index. Here we are using VectorStore Index, which is also one of the most common types of indexing.

In [39]:
# Set the OpenAI API key from environment variables
openai.api_key = os.environ["OPENAI_API_KEY"]

# Create a default service context for the OpenAI model with the specified temperature
# The 'temperature' parameter controls the randomness in the model's responses (0 means deterministic, 1 means maximum randomness)
service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0) # degree of randomness from 0 to 1. 
)

# Create a GPTVectorStoreIndex from a set of documents using the service context
# This method indexes the documents (docs) so they can be searched or queried using the vector-based GPT model.
index = GPTVectorStoreIndex.from_documents(documents=docs, service_context=service_context)

In [40]:
# Save (persist) the current state of the index to a specified directory.
# The index will be stored in the "./data/index.vecstore" directory, allowing it to be reloaded later without rebuilding.
index.storage_context.persist(persist_dir="./data/index.vecstore")  

## Train Questions Dataset
Manually created 25 questions that users are likely to ask and allocated them as the train questions dataset.

In [43]:
# Putting the 25 questions into a list called train_questions
train_questions = [
"Which streaming platform between Netflix, Disney+ and Amazon Prime Video should be chosen looking only at the number of Asian movies and shows available for streaming?",
"Which streaming service platform offers a better selection of documentaries only, Netflix, Disney+ or Amazon Prime Video?",
"Between Netflix, Disney+ and Amazon Prime Video, which streaming platform has the highest number of original shows and movies?",
"Looking at the selection of old movies and shows before year 2000 only, which streaming platform between Netflix, Disney+ and Amazon Prime Video should be chosen?",
"Which streaming service platform, Netflix, Disney+ or Amazon Prime Video, has the greatest number of users?",
"In terms of the quantity of Japanese anime only, should Netflix, Disney+ or Amazon Prime Video be chosen?",
"Based on the availability of superhero movies and shows only, which streaming platform between Netflix, Disney+ or Amazon Prime Video has the greatest number of superhero movies and shows?",
"In terms of US animation content only, which streaming service platform offers the best variety of US animation content, Netflix, Disney+ or Amazon Prime Video?",
"Depending on highest average IMDB rating only for its shows and movies, which streaming platform among Netflix, Disney+ and Amazon Prime Video should be chosen?",
"Which streaming service platform has the highest number of total shows and movies available on the platform?",
"How does Netflix compare to Disney+ and Amazon Prime Video in terms of the number of Asian movies and dramas available on the platform?",
"Which streaming service platform offers the most original shows and movies, Netflix, Disney+ or Amazon Prime Video?",
"Are there more Japanese anime options on Netflix, Disney+ or Amazon Prime Video?",
"How does the availability of US animation content differ between Netflix, Disney+ and Amazon Prime Video?",
"Which streaming service platform has the highest number of superhero movies and shows?",
"Is there a significant difference in the number of documentaries available on Netflix, Disney+ and Amazon Prime Video?",
"How do the IMDB ratings of content on Netflix, Disney+ and Amazon Prime Video compare, depending on the highest average IMDB rating?",
"Considering my preference for old movies and shows before year 2000, which platform Netflix, Disney+ or Amazon Prime Video offers the most options?",
"With a budget of $7, which streaming platform Netflix, Disney+ or Amazon Prime Video should be chosen based on their price?",
"Based on the platform’s popularity like the largest number of users, which streaming service platform should be chosen?",
"Considering my preference for original shows and movies, which platform has the highest number of original shows and movies?",
"Japanese anime aligns with my interest and preference, which platform offers the most Japanese anime?",
"Which streaming platform Netflix, Disney+ or Amazon Prime Video cater specifically to my preference for Asian movies and shows?",
"Looking at the price between streaming platform Netflix, Disney+ and Amazon Prime Video, which streaming platform should be chosen with a budget of $8?",
"Which streaming platform between Netflix, Disney+ and Amazon Prime Video offers the most documentaries?"]          

In [44]:
# writes a list of all 25 questions to a file named train_questions.txt and prints each question to the console.
with open("train_questions.txt", "w") as f:
    for question in train_questions:
        f.write(question + "\n")
        print(question)

Which streaming platform between Netflix, Disney+ and Amazon Prime Video should be chosen looking only at the number of Asian movies and shows available for streaming?
Which streaming service platform offers a better selection of documentaries only, Netflix, Disney+ or Amazon Prime Video?
Between Netflix, Disney+ and Amazon Prime Video, which streaming platform has the highest number of original shows and movies?
Looking at the selection of old movies and shows before year 2000 only, which streaming platform between Netflix, Disney+ and Amazon Prime Video should be chosen?
Which streaming service platform, Netflix, Disney+ or Amazon Prime Video, has the greatest number of users?
In terms of the quantity of Japanese anime only, should Netflix, Disney+ or Amazon Prime Video be chosen?
Based on the availability of superhero movies and shows only, which streaming platform between Netflix, Disney+ or Amazon Prime Video has the greatest number of superhero movies and shows?
In terms of US an

## Evaluation Questions Dataset
Using the 25 train questions created earlier, rephrase, amend and shuffle them to create another set of 25 questions that are slightly different. Then allocate these new 25 questions as the evaluation questions dataset.

In [47]:
# Putting the 25 rephrased, amended and shuffled questions into a list called eval_questions
eval_questions = [
"With a budget of $8, which streaming service—Netflix, Disney+, or Amazon Prime Video—offers the best price option?",
"If I'm interested in movies and shows from before 2000, which platform—Netflix, Disney+, or Amazon Prime Video—has the most extensive collection?",
"How does Netflix's library of Asian shows and movies stack up against Disney+ and Amazon Prime Video?",
"For Japanese anime, which platform—Netflix, Disney+, or Amazon Prime Video—offers the largest collection of titles?",
"Which platform—Netflix, Disney+, or Amazon Prime Video—features the most superhero films and series?",
"Which platform, Netflix, Disney+, or Amazon Prime Video, offers the most original programming?",
"If I’m looking for older films and shows from before the year 2000, which platform—Netflix, Disney+, or Amazon Prime Video—would be the best option?",
"Which platform—Netflix, Disney+, or Amazon Prime Video—has the biggest selection of documentaries?",
"For U.S. animated content, which streaming platform—Netflix, Disney+, or Amazon Prime Video—offers the most variety?",
"How do Netflix, Disney+, and Amazon Prime Video compare in terms of their average IMDb ratings?",
"With a budget of $7, which streaming service—Netflix, Disney+, or Amazon Prime Video—offers the most affordable option?",
"Out of Netflix, Disney+, and Amazon Prime Video, which service has the greatest number of original movies and series?",
"Between Netflix, Disney+, and Amazon Prime Video, which one has the most superhero movies and shows?",
"For original series and films, which service—Netflix, Disney+, or Amazon Prime Video—has the largest offering?",
"Based on user count, which platform—Netflix, Disney+, or Amazon Prime Video—should be selected?",
"When it comes to documentaries, which streaming platform—Netflix, Disney+, or Amazon Prime Video—provides the top selection?",
"Among Netflix, Disney+, and Amazon Prime Video, which streaming service provides the most Japanese anime content?",
"If I’m into Japanese anime, which platform has the broadest selection?",
"Based on average IMDb ratings, which platform—Netflix, Disney+, or Amazon Prime Video—is the better choice?",
"Is there a significant difference in the number of documentaries available on Netflix, Disney+, and Amazon Prime Video?",
"Which of Netflix, Disney+, or Amazon Prime Video features the largest collection of Asian films and shows?",
"Which streaming service has the highest number of users: Netflix, Disney+, or Amazon Prime Video?",
"Which service—Netflix, Disney+, or Amazon Prime Video—boasts the biggest collection of movies and series?",
"Which service—Netflix, Disney+, or Amazon Prime Video—is the best fit for someone who prefers Asian films and shows?",
"How do Netflix, Disney+, and Amazon Prime Video differ when it comes to U.S. animated content?"]       

In [49]:
#writes a list of all 25 questions to a file named eval_questions.txt
with open("eval_questions.txt", "w") as f:
    for question in eval_questions:
        f.write(question + "\n")
        print(question)

With a budget of $8, which streaming service—Netflix, Disney+, or Amazon Prime Video—offers the best price option?
If I'm interested in movies and shows from before 2000, which platform—Netflix, Disney+, or Amazon Prime Video—has the most extensive collection?
How does Netflix's library of Asian shows and movies stack up against Disney+ and Amazon Prime Video?
For Japanese anime, which platform—Netflix, Disney+, or Amazon Prime Video—offers the largest collection of titles?
Which platform—Netflix, Disney+, or Amazon Prime Video—features the most superhero films and series?
Which platform, Netflix, Disney+, or Amazon Prime Video, offers the most original programming?
If I’m looking for older films and shows from before the year 2000, which platform—Netflix, Disney+, or Amazon Prime Video—would be the best option?
Which platform—Netflix, Disney+, or Amazon Prime Video—has the biggest selection of documentaries?
For U.S. animated content, which streaming platform—Netflix, Disney+, or Amaz

In [50]:
#prints the total number of documents in the docs list.
print("Total number of documents:", len(docs))

Total number of documents: 25


## Initial Evaluation

For this evaluation with GPT-3.5 Query Engine, we will be using the [`ragas` evaluation library](https://github.com/explodinggradients/ragas).

For this notebook, we will be using the following two metrics:

- `answer_relevancy` - This measures how relevant the generated answer is to the prompt. If the answer is incomplete or contains redundant information, the score will be low. Relevance is quantified by comparing the generated answer to an expected reference answer. The score typically ranges from 0 to 1, with higher values indicating better relevance.

- `faithfulness` - This measures the factual consistency of the generated answer with the given context. It is assessed through a multi-step process that involves extracting statements from the generated answer and verifying each one against the context. The final score is scaled between 0 and 1, with higher scores indicating greater factual consistency.

In [ ]:
# Set a random seed to ensure reproducibility of the shuffling process
random.seed(42)

# Shuffle the list of documents (docs) in-place to randomize their order
random.shuffle(docs)

# Create a new service context for the OpenAI model (gpt-3.5-turbo) with a temperature of 0
# The temperature is set to 0 for deterministic output from the model
gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0)
)

In [58]:
#reads questions from a file named eval_questions.txt and stores them in a list named questions.
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [60]:
# Input the model to be used "gpt-3.5-turbo"
# Limit the context window of the model to 2048 tokens, ensuring that the "refine" mechanism is used for longer texts.
gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0), context_window=2048
)

# Create a vector store index from the provided documents (docs) using the specified service context (gpt_context).
# This index will be used for semantic similarity searches.
index = VectorStoreIndex.from_documents(docs, service_context=gpt_context)

# Create a query engine from the index that returns the top 2 most semantically similar documents for a given query.
query_engine = index.as_query_engine(similarity_top_k=2)

# The GPT-3.5 turbo model is used to understand the semantic content of documents and retrieve those most similar to the given query.

In [61]:
# Initialize empty lists to store contexts (relevant document sections) and answers for each question.
contexts = []
answers = []

# Loop through each question in the 'questions' list.
for question in questions:
    # Use the query engine to find the most relevant documents and generate an answer for the question.
    response = query_engine.query(question)

    # Extract the content of the source nodes (relevant document sections) from the response
    # and append them as context for the question.
    contexts.append([x.node.get_content() for x in response.source_nodes])

    # Append the generated answer (as a string) to the 'answers' list.
    answers.append(str(response))

#store the contexts and answers of the responses

In [63]:
# Retrieve the first 25 questions from the 'questions' list.
questions[:25]

['With a budget of $8, which streaming service—Netflix, Disney+, or Amazon Prime Video—offers the best price option?',
 "If I'm interested in movies and shows from before 2000, which platform—Netflix, Disney+, or Amazon Prime Video—has the most extensive collection?",
 "How does Netflix's library of Asian shows and movies stack up against Disney+ and Amazon Prime Video?",
 'For Japanese anime, which platform—Netflix, Disney+, or Amazon Prime Video—offers the largest collection of titles?',
 'Which platform—Netflix, Disney+, or Amazon Prime Video—features the most superhero films and series?',
 'Which platform, Netflix, Disney+, or Amazon Prime Video, offers the most original programming?',
 'If I’m looking for older films and shows from before the year 2000, which platform—Netflix, Disney+, or Amazon Prime Video—would be the best option?',
 'Which platform—Netflix, Disney+, or Amazon Prime Video—has the biggest selection of documentaries?',
 'For U.S. animated content, which streaming 

In [64]:
# Create a dataset from the 'questions', 'answers', and 'contexts' lists by converting them into a dictionary format.
# Each key represents a column, with "question", "answer", and "contexts" being the dataset fields.
ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

# Evaluate the dataset (ds) using the specified evaluation metrics: 'answer_relevancy' and 'faithfulness'.
# These metrics will assess the quality of the generated answers.
result = evaluate(ds,[answer_relevancy, faithfulness])

# Print the result of the evaluation, which includes scores for both metrics.
print(result)

# Evaluate the answer_relevancy & faithfulness using ragas. Ragas score  = (answer_relevancy + faithfulness) / 2

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'answer_relevancy': 0.9379, 'faithfulness': 0.9333}


## 2nd Evaluation

For this evaluation with GPT-4 Query Engine, we will be using the [`ragas` evaluation library](https://github.com/explodinggradients/ragas).

For this notebook, we will be using the following two metrics:

- `answer_relevancy` - This measures how relevant the generated answer is to the prompt. If the answer is incomplete or contains redundant information, the score will be low. Relevance is quantified by comparing the generated answer to an expected reference answer. The score typically ranges from 0 to 1, with higher values indicating better relevance.

- `faithfulness` - This measures the factual consistency of the generated answer with the given context. It is assessed through a multi-step process that involves extracting statements from the generated answer and verifying each one against the context. The final score is scaled between 0 and 1, with higher scores indicating greater factual consistency.

In [66]:
#reads questions from a file named eval_questions.txt and stores them in a list named questions.
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [67]:
# Input the model to be used "gpt-4"
# Limit the context window of the model to 2048 tokens, ensuring that the "refine" mechanism is used for longer texts.
gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0), context_window=2048
)

# Create a vector store index from the provided documents (docs) using the specified service context (gpt_context).
# This index will be used for semantic similarity searches.
index = VectorStoreIndex.from_documents(docs, service_context=gpt_context)

# Create a query engine from the index that returns the top 2 most semantically similar documents for a given query.
query_engine = index.as_query_engine(similarity_top_k=2)

# The GPT-4 model is used to understand the semantic content of documents and retrieve those most similar to the given query.

In [68]:
# Initialize empty lists to store contexts (relevant document sections) and answers for each question.
contexts = []
answers = []

# Loop through each question in the 'questions' list.
for question in questions:
    # Use the query engine to find the most relevant documents and generate an answer for the question.
    response = query_engine.query(question)

    # Extract the content of the source nodes (relevant document sections) from the response
    # and append them as context for the question.
    contexts.append([x.node.get_content() for x in response.source_nodes])

    # Append the generated answer (as a string) to the 'answers' list.
    answers.append(str(response))
    
#store the contexts and answers of the responses

In [69]:
# Retrieve the first 25 questions from the 'questions' list.
questions[:25]

['With a budget of $8, which streaming service—Netflix, Disney+, or Amazon Prime Video—offers the best price option?',
 "If I'm interested in movies and shows from before 2000, which platform—Netflix, Disney+, or Amazon Prime Video—has the most extensive collection?",
 "How does Netflix's library of Asian shows and movies stack up against Disney+ and Amazon Prime Video?",
 'For Japanese anime, which platform—Netflix, Disney+, or Amazon Prime Video—offers the largest collection of titles?',
 'Which platform—Netflix, Disney+, or Amazon Prime Video—features the most superhero films and series?',
 'Which platform, Netflix, Disney+, or Amazon Prime Video, offers the most original programming?',
 'If I’m looking for older films and shows from before the year 2000, which platform—Netflix, Disney+, or Amazon Prime Video—would be the best option?',
 'Which platform—Netflix, Disney+, or Amazon Prime Video—has the biggest selection of documentaries?',
 'For U.S. animated content, which streaming 

In [70]:
# Create a dataset from the 'questions', 'answers', and 'contexts' lists by converting them into a dictionary format.
# Each key represents a column, with "question", "answer", and "contexts" being the dataset fields.
ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

# Evaluate the dataset (ds) using the specified evaluation metrics: 'answer_relevancy' and 'faithfulness'.
# These metrics will assess the quality of the generated answers.
result = evaluate(ds,[answer_relevancy, faithfulness])

# Print the result of the evaluation, which includes scores for both metrics.
print(result)

#evaluate result on answer_relevancy & faithfulness using ragas. Ragas score  = (answer_relevancy + faithfulness) / 2

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'answer_relevancy': 0.9395, 'faithfulness': 0.9700}


## 3rd Evaluation (Fine Tune)

For this evaluation with fine tuned ft:gpt-3.5-turbo-1106 Query Engine, we will be using the [`ragas` evaluation library](https://github.com/explodinggradients/ragas).

For this notebook, we will be using the following two metrics:

- `answer_relevancy` - This measures how relevant the generated answer is to the prompt. If the answer is incomplete or contains redundant information, the score will be low. Relevance is quantified by comparing the generated answer to an expected reference answer. The score typically ranges from 0 to 1, with higher values indicating better relevance.

- `faithfulness` - This measures the factual consistency of the generated answer with the given context. It is assessed through a multi-step process that involves extracting statements from the generated answer and verifying each one against the context. The final score is scaled between 0 and 1, with higher scores indicating greater factual consistency.

The purpose of running this fine tuned ft:gpt-3.5-turbo-1106 model is to check and see if the fine tuned model can achive greater values for its answer relevancy as well as its faithfulness.etter.

### Generate Training Data

Here, we use GPT-3.5-turbo and the `OpenAIFineTuningHandler` to collect data that we want to train on.

In [72]:
# Initialize the fine-tuning handler for OpenAI, which will manage the fine-tuning process.
finetuning_handler = OpenAIFineTuningHandler()

# Create a callback manager to handle events and callbacks during the fine-tuning process.
# The callback manager includes the fine-tuning handler.
callback_manager = CallbackManager([finetuning_handler])

# Create a service context for the GPT-3.5-turbo model.
# Set the context window to 2048 tokens to limit the amount of text processed, which helps test the refine process.
# Associate the callback manager with the service context to handle fine-tuning callbacks.
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

# The GPT-3.5-turbo model is configured to understand and process the semantic content of documents,
# and will use this understanding to find documents semantically similar to a given query.

In [73]:
# Create a VectorStoreIndex from the provided collection of documents (docs) using the specified service context (gpt_35_context).
# This index will be used to perform semantic searches on the documents.
index = VectorStoreIndex.from_documents(docs, service_context=gpt_35_context)

# Convert the created VectorStoreIndex into a query engine that can perform similarity searches.
# The query engine is configured to return the top 2 most similar documents for any given query.
query_engine = index.as_query_engine(similarity_top_k=2) 

In [74]:
#reads questions from a file named train_questions.txt and stores them in a list named questions.
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [75]:
# loop that iterates over a list of questions, querying a query_engine for each question and storing the response in a variable named response
for question in questions:
    response = query_engine.query(question)

### Create Fine Tuned Engine

In [76]:
#save fine-tuning events to a JSONL file called finetune.
finetuning_handler.save_finetuning_events("finetune.jsonl")

Wrote 25 examples to finetune.jsonl


### Evaluating Fine Tuned Engine

After some time, your model will be done training!

The next step is running our fine-tuned model on our evaluation dataset again to measure any performance increase.se.

In [78]:
#reads questions from a file named eval_questions.txt and stores them in a list named questions.
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [79]:
# Input the model to be used "ft:gpt-3.5-turbo-1106:personal::9yLUndRk"
# Limit the context window of the model to 2048 tokens, ensuring that the "refine" mechanism is used for longer texts.
ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model="ft:gpt-3.5-turbo-1106:personal::9yLUndRk",temperature=0, openai_api_key=openai.api_key), context_window=2048
)

# Create a vector store index from the provided documents (docs) using the specified service context (gpt_context).
# This index will be used for semantic similarity searches.
index = VectorStoreIndex.from_documents(docs, service_context=ft_context)

# Create a query engine from the index that returns the top 2 most semantically similar documents for a given query.
query_engine = index.as_query_engine(similarity_top_k=2)

# The fine tuned gpt-3.5-turbo-1106 model is used to understand the semantic content of documents and retrieve those most similar to the given query. 

In [80]:
# Initialize empty lists to store contexts (relevant document sections) and answers for each question.
contexts = []
answers = []

# Loop through each question in the 'questions' list.
for question in questions:
    # Use the query engine to find the most relevant documents and generate an answer for the question.
    response = query_engine.query(question)
    
    # Extract the content of the source nodes (relevant document sections) from the response
    # and append them as context for the question.
    contexts.append([x.node.get_content() for x in response.source_nodes])

    # Append the generated answer (as a string) to the 'answers' list.
    answers.append(str(response))
    
#store the contexts and answers of the responses

In [81]:
# Create a dataset from the 'questions', 'answers', and 'contexts' lists by converting them into a dictionary format.
# Each key represents a column, with "question", "answer", and "contexts" being the dataset fields.
ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

# Evaluate the dataset (ds) using the specified evaluation metrics: 'answer_relevancy' and 'faithfulness'.
# These metrics will assess the quality of the generated answers.
result = evaluate(ds, [answer_relevancy, faithfulness])

# Print the result of the evaluation, which includes scores for both metrics.
print(result)

#evaluate result on answer_relevancy & faithfulness using ragas. Ragas score  = (answer_relevancy + faithfulness) / 2

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'answer_relevancy': 0.9419, 'faithfulness': 0.9300}


#### Previous Results
|         Models        | answer_relevancy | faithfulness |
|:---------------------:|:----------------:|:------------:|
| gpt-3.5-turbo         | 0.8734           | 0.1894       |
| ft:gpt-3.5-turbo-1106 | 0.8892           | 0.2176       |
| gpt-4                 | 0.6523           | 0.5279       |

Previously when running the gpt models on the same user preference dataset used in the classification modelling, I got the relevancy and faithfulness results that is seen above. After trying multiple attmepts and methods of trying to improve the scores, such as changing the data in the user preference dataset and even making the train and evaluation questions as direct as possible, the scores still would not increase.

After researching, I had found out that because the user preference dataset as well as the raw data used to create the user preference dataset was based and collected in 2023, while the gpt models are using the lastest 2024 database. This resulted in a discrepancy between the gpt model's database and the user preference dataset used. Hence, this resulted in the low faithfulness.

To resolve this, I would have to update my user preference dataset as well as the raw data collected to the present(2024) and would have to constantly update them to keep the model accurate.

#### Latest Results
| Models                | answer_relevancy | faithfulness |
|-----------------------|------------------|--------------|
| gpt-3.5-turbo         | 0.9379           | 0.9333       |
| ft:gpt-3.5-turbo-1106 | 0.9419           | 0.9300       |
| **gpt-4**             | **0.9395**       | **0.9700**   |

In the end, I had to change my approach and use a dataset filled with many likely questions that user will ask and accurate answers to those questions, called questions_answers. By using a different dataset, I had gotten the results you see above.

In conclustion: looking at the latest results, the best model to use would be the gpt-4 model. This is because the answer relevancy is very similar to the others at 0.9395 but the faithfulness is much higher than the other 2 models at 0.97. Hence, the gpt-4 model would be the best in this case.